# Download Data

In [12]:
import torch
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
path_data = './data'
transform = transforms.ToTensor()

train_data = FashionMNIST(root=path_data, download=True, train=True, transform=transform)
test_data = FashionMNIST(root=path_data, download=True, train=False, transform=transform)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
          'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [3]:
batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Build Model

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # in shape=(1, 28, 28), out shape=(10, 26, 26)  (zero-padding, i.e. valid)
        self.conv1 = nn.Conv2d(1, 10, 3)
        
        self.pool = nn.MaxPool2d(2, stride=2)
        
        # in shape=(10, 13, 13), out shape=(20, 11, 11)
        self.conv2 = nn.Conv2d(10, 20, 3)
        
        # in shape=(20*5*5), out shape=(10)
        self.fc1 = nn.Linear(20*5*5, 10)
        
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        
        x = F.log_softmax(x, dim=1)
        
        return x
        
        
net = CNN()
print(net)
for p in net.parameters():
    print(p.shape)

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=500, out_features=10, bias=True)
)
torch.Size([10, 1, 3, 3])
torch.Size([10])
torch.Size([20, 10, 3, 3])
torch.Size([20])
torch.Size([10, 500])
torch.Size([10])


# Define Loss and Optimizer

In [5]:
import torch.optim as optim

# Negative Log Likelihood Loss (NLL Loss)
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

# Define Train process

In [8]:
def train(n_epochs):
    print_period = 1000
    loss_over_time = []
    
    for epoch in range(n_epochs):
        running_loss = 0.0
        
        for index_batch, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad()
            
            
            y_hat = net.forward(inputs)
            
            loss = criterion(y_hat, labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
            
            if index_batch % print_period == (print_period - 1):
                avg_loss = running_loss / print_period
                loss_over_time.append(avg_loss)
                
                print('Epoch:{}, Batch:{}, Avg loss:{}'.format(
                    epoch, index_batch, avg_loss
                ))
                
                running_loss = 0.0
                
    print('Finished Training')
    return loss_over_time

In [9]:
epochs = 30
training_loss = train(epochs)

Epoch:0, Batch:999, Avg loss:2.2889203345775604
Epoch:1, Batch:999, Avg loss:2.178057375192642
Epoch:2, Batch:999, Avg loss:1.7991369761228562
Epoch:3, Batch:999, Avg loss:1.5904617938995362
Epoch:4, Batch:999, Avg loss:1.528907211780548
Epoch:5, Batch:999, Avg loss:1.5076966230273248
Epoch:6, Batch:999, Avg loss:1.400166346013546
Epoch:7, Batch:999, Avg loss:1.3373704303503036
Epoch:8, Batch:999, Avg loss:1.3072202153801917
Epoch:9, Batch:999, Avg loss:1.3075704311728478
Epoch:10, Batch:999, Avg loss:1.2877609106302261
Epoch:11, Batch:999, Avg loss:1.275410022199154
Epoch:12, Batch:999, Avg loss:1.2631211723685265
Epoch:13, Batch:999, Avg loss:1.252165415227413
Epoch:14, Batch:999, Avg loss:1.2439380726218223
Epoch:15, Batch:999, Avg loss:1.2386520525217057
Epoch:16, Batch:999, Avg loss:1.2294009672999382
Epoch:17, Batch:999, Avg loss:1.2256276106238366
Epoch:18, Batch:999, Avg loss:1.2179107248187064
Epoch:19, Batch:999, Avg loss:1.213337930560112
Epoch:20, Batch:999, Avg loss:1.2062

# Define Validation Process

In [22]:
def valid():
    test_loss = torch.zeros(1)
    
    class_correct = list(0. for _ in range(10))
    class_total = list(0. for _ in range(10))
    
    
    for batch_i, (inputs, labels) in enumerate(test_loader):
        
        outputs = net.forward(inputs)
        
        loss = criterion(outputs, labels)
        
        # running average
        test_loss = test_loss + ((torch.ones(1) / (batch_i + 1)) * (loss.data - test_loss))
        
        _, predicted = torch.max(outputs.data, 1)
        
        correct = np.squeeze(predicted.eq(labels.data.view_as(predicted)))
        
        for i in range(10):
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            
            
    print('Test loss:{:.6f}'.format(test_loss.numpy()[0]))
        
        
    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of {}: {} ({}/{})'.format(
                classes[i], 100. * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])
            ))
        else:
            print('Test Accuracy of {}: N/A (No training examples)'.format(classes[i]))
        
        
    print('Test Accuracy (Overall): {} ({:2.1f}/{:2.1f})'.format(
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)
    ))
    
    
valid()

Test loss:1.190068
Test Accuracy of T-shirt/top: 88.70431893687707 (267.0/301.0)
Test Accuracy of Trouser: 90.82278481012658 (287.0/316.0)
Test Accuracy of Pullover: 0.0 (0.0/345.0)
Test Accuracy of Dress: 93.67088607594937 (296.0/316.0)
Test Accuracy of Coat: 0.0 (0.0/310.0)
Test Accuracy of Sandal: 93.86503067484662 (306.0/326.0)
Test Accuracy of Shirt: 0.0 (0.0/311.0)
Test Accuracy of Sneaker: 94.01993355481727 (283.0/301.0)
Test Accuracy of Bag: 93.64548494983278 (280.0/299.0)
Test Accuracy of Ankle boot: 0.0 (0.0/305.0)
Test Accuracy (Overall): 54.92012779552716 (1719.0/3130.0)
